In [2]:
import pandas as pd
import numpy as np
import os
import keras
import sklearn
import json
import requests
import itertools
import datetime
import math

import seaborn as sns

from scipy import linalg
from scipy.spatial.distance import pdist

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder

from streetaddress import StreetAddressFormatter, StreetAddressParser

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

Using TensorFlow backend.


In [6]:
datadir = '../data/plows/v1/clean'
files = os.listdir(datadir)
#gc_key = os.environ['GEOCODING_SECRET_KEY']
file = 'all_plows.csv'
address = '700 Stinard Avenue, Syracuse, NY 13207'
addr_parser = StreetAddressParser()

In [7]:
#res = requests.get('https://maps.googleapis.com/maps/api/reverse_geocode/json?address={}&key={}'.format(address,gc_key))
#results = res.json()['results'][0]
#lat = results['geometry']['location']['lat']
#long = results['geometry']['location']['lng']

In [11]:
df = pd.read_csv(os.path.join(datadir, file))
df


,activity_type,address,latitude,longitude,repair_type,strHeading,truck_name,time,date
0,Stop Moving,306 SENECA TNPK E TO ARSENAL ...,42.999824,-76.138107,SNOW & ICE,E,178.0,00:00:07,01/08/2018
1,,106 ARSENAL DR ...,42.999739,-76.138110,SNOW & ICE,E,178.0,00:01:07,01/08/2018
2,,306 SENECA TNPK E TO ARSENAL ...,42.999786,-76.138090,SNOW & ICE,E,178.0,00:02:07,01/08/2018
3,,121 ARSENAL DR ...,42.999810,-76.137820,SNOW & ICE,E,178.0,00:03:07,01/08/2018
4,Start Moving,121 ARSENAL DR ...,42.999808,-76.137770,SNOW & ICE,E,178.0,00:03:09,01/08/2018
5,Stop Moving,137 ARSENAL DR ...,42.999789,-76.137208,SNOW & ICE,E,178.0,00:03:27,01/08/2018
6,,131 ARSENAL DR ...,42.999742,-76.137425,SNOW & ICE,W,178.0,00:04:23,01/08/2018
7,Start Moving,131 ARSENAL DR ...,42.999746,-76.137481,SNOW & ICE,W,178.0,00:04:26,01/08/2018
8,Stop Moving,302 SENECA TNPK E & MONTICELL ...,42.999799,-76.138434,SNOW & ICE,W,178.0,00:04:59,01/08/2018
9,,302 SENECA TNPK E & MONTICELL ...,42.999829,-76.138618,SNOW & ICE,N,178.0,00:05:31,01/08/2018


In [9]:
def get_street(address):
    return addr_parser.parse(address)['street_full']
def get_street_num(address):
    return addr_parser.parse(address)['house']

In [12]:
df.dropna(inplace=True)
df.assign(street=lambda x: x['address'].apply(get_street), street_num=lambda x: x['address'].apply(get_street_num), inplace=True)

,activity_type,address,latitude,longitude,repair_type,strHeading,truck_name,time,date,street,street_num,inplace
0,Stop Moving,306 SENECA TNPK E TO ARSENAL ...,42.999824,-76.138107,SNOW & ICE,E,178.0,00:00:07,01/08/2018,SENECA TNPK E TO ARSENAL,306,True
1,,106 ARSENAL DR ...,42.999739,-76.138110,SNOW & ICE,E,178.0,00:01:07,01/08/2018,ARSENAL DR,106,True
2,,306 SENECA TNPK E TO ARSENAL ...,42.999786,-76.138090,SNOW & ICE,E,178.0,00:02:07,01/08/2018,SENECA TNPK E TO ARSENAL,306,True
3,,121 ARSENAL DR ...,42.999810,-76.137820,SNOW & ICE,E,178.0,00:03:07,01/08/2018,ARSENAL DR,121,True
4,Start Moving,121 ARSENAL DR ...,42.999808,-76.137770,SNOW & ICE,E,178.0,00:03:09,01/08/2018,ARSENAL DR,121,True
5,Stop Moving,137 ARSENAL DR ...,42.999789,-76.137208,SNOW & ICE,E,178.0,00:03:27,01/08/2018,ARSENAL DR,137,True
6,,131 ARSENAL DR ...,42.999742,-76.137425,SNOW & ICE,W,178.0,00:04:23,01/08/2018,ARSENAL DR,131,True
7,Start Moving,131 ARSENAL DR ...,42.999746,-76.137481,SNOW & ICE,W,178.0,00:04:26,01/08/2018,ARSENAL DR,131,True
8,Stop Moving,302 SENECA TNPK E & MONTICELL ...,42.999799,-76.138434,SNOW & ICE,W,178.0,00:04:59,01/08/2018,SENECA TNPK E & MONTICELL,302,True
9,,302 SENECA TNPK E & MONTICELL ...,42.999829,-76.138618,SNOW & ICE,N,178.0,00:05:31,01/08/2018,SENECA TNPK E & MONTICELL,302,True


In [13]:
df.to_csv('../data/plows/v1/clean/all_plows.csv')

In [30]:
trainX = df[['longitude', 'latitude']][:int(df.shape[0]/2)].iloc[::10, :].values
testX = df[['longitude', 'latitude']][int(df.shape[0]/2):].iloc[::10, :].values
clusters = 5

In [ ]:
model = GaussianMixture(n_components=clusters)
model.fit(trainX)

In [ ]:
y_pred = model.predict(testX)
model.bic(testX)

In [ ]:
colors = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, clusters)]

In [ ]:
def plot_results(X, Y_, means, covariances,  title):
    data = []
    for i, (mean, covar, color) in enumerate(zip(
            means, covariances, colors)):
        
        v, w = linalg.eigh(covar)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
        trace = go.Scatter(x=X[Y_ == i, 0], y=X[Y_ == i, 1], 
                           mode='markers',
                           marker=dict(color=color))
        data.append(trace)
        # Plot an ellipse to show the Gaussian component
        a =  v[1]
        b =  v[0]
        x_origin = mean[0]
        y_origin = mean[1]
        x_ = [ ]
        y_ = [ ]
    
        for t in range(0,361,10):
            x = a*(math.cos(math.radians(t))) + x_origin
            x_.append(x)
            y = b*(math.sin(math.radians(t))) + y_origin
            y_.append(y)
    
        elle = go.Scatter(x=x_ , y=y_, mode='lines',
                          showlegend=False,
                          line=dict(color=color,
                                   width=2))
        data.append(elle)
       
    layout = go.Layout(title=title, showlegend=False,
                       xaxis=dict(zeroline=False, showgrid=False),
                       yaxis=dict(zeroline=False, showgrid=False),)
    fig = go.Figure(data=data, layout=layout)
    
    return fig

In [ ]:
fig = plot_results(testX, y_pred, model.means_, model.covariances_, 'Gaussian Mixture')
iplot(fig)

In [ ]:
df['activity_type'].unique()

In [ ]:
trucks = df['truck_name'].unique()
len(trucks)

In [ ]:
df[]